In [ ]:
!pip install -q parlai

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 145 kB 62.9 MB/s 
     |████████████████████████████████| 181 kB 55.3 MB/s 
     |████████████████████████████████| 125 kB 62.4 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 2.8 MB 54.6 MB/s 
     |████████████████████████████████| 40 kB 5.4 MB/s 
     |████████████████████████████████| 6.5 MB 54.0 MB/s 
     |████████████████████████████████| 248 kB 51.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 15.7 MB 56.5 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 48 kB 5.5 MB/s 
     |█████████████████████████

In [ ]:
from parlai.scripts.interactive import Interactive

Interactive.main(
    model_file='zoo:dodecadialogue/empathetic_dialogues_ft/model',
    # task="empathetic_dialogues"
)

19:09:27 | building data: /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/dodecadialogue_v2.tgz
19:09:27 | Downloading http://parl.ai/downloads/_models/dodecadialogue/dodecadialogue_v2.tgz to /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/dodecadialogue_v2.tgz


19:09:28 | building data: /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/empathetic_dialogues_ft.tgz
19:09:28 | Downloading http://parl.ai/downloads/_models/dodecadialogue/empathetic_dialogues_ft/empathetic_dialogues_ft.tgz to /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/empathetic_dialogues_ft.tgz


19:10:07 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model (previously: data/models/dodecadialogue/empathetic_dialogues/model)
19:10:07 | Using CUDA
19:10:07 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model.dict
19:10:07 | num words = 54946
19:10:07 | ImageSeq2seq: full interactive mode on.
19:10:09 | Total parameters: 88,559,104 (88,559,104 trainable)
19:10:09 | Loading existing model params from /usr/local/lib/python3.7/dist-packages/data/models/dodecadialogue/empathetic_dialogues_ft/model
19:10:10 | Opt:
19:10:10 |     activation: gelu
19:10:10 |     adafactor_eps: '(1e-30, 0.001)'
19:10:10 |     adam_eps: 1e-08
19:10:10 |     add_p1_after_newln: False
19:10:10 |     aggregate_micro: False
19:10:10 |     allow_missing_init_opts: False
19:10:10 |     attention_dropout: 0.0
19:10:10 |     batch_length_range: 5
19:10:10 |     batch_sor

KeyboardInterrupt: ignored

# Imp chunk teacher


In [ ]:
from typing import Optional
from parlai.core.params import ParlaiParser
from parlai.core.opt import Opt
import torch.nn as nn
import torch.nn.functional as F
import parlai.core.torch_generator_agent as tga
from parlai.core.agents import register_agent, Agent


class Encoder(nn.Module):
    def __init__(self, embeddings, hidden_size):
        super().__init__()

        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )

    def forward(self, input_tokens):
        embedded = self.embeddings(input_tokens)
        _output, hidden = self.lstm(embedded)
        return hidden


class Decoder(nn.Module):
    def __init__(self, embeddings, hidden_size):
        super().__init__()
        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )

    def forward(self, input, encoder_state, incr_state=None):
        embedded = self.embeddings(input)

        if incr_state is None:
            state = encoder_state
        else:
            state = incr_state

        output, incr_state = self.lstm(embedded, state)

        return output, incr_state


class ExampleModel(tga.TorchGeneratorModel):
    def __init__(self, dictionary, hidden_size=1024):
        super().__init__(
            padding_idx=dictionary[dictionary.null_token],
            start_idx=dictionary[dictionary.start_token],
            end_idx=dictionary[dictionary.end_token],
            unknown_idx=dictionary[dictionary.unk_token],
        )
        self.embeddings = nn.Embedding(len(dictionary), hidden_size)
        self.encoder = Encoder(self.embeddings, hidden_size)
        self.decoder = Decoder(self.embeddings, hidden_size)

    def output(self, decoder_output):
        return F.linear(decoder_output, self.embeddings.weight)

    def reorder_encoder_states(self, encoder_states, indices):
        h, c = encoder_states
        return h[:, indices, :], c[:, indices, :]

    def reorder_decoder_incremental_state(self, incr_state, indices):
        """
        Reorder the decoder states to select only the given batch indices.
        This method can be a stub which always returns None; this will result in the
        decoder doing a complete forward pass for every single token, making generation
        O(n^2). However, if any state can be cached, then this method should be
        implemented to reduce the generation complexity to O(n).
        """
        h, c = incr_state
        return h[:, indices, :], c[:, indices, :]

@register_agent("my_first_lstm")
class Seq2seqAgent(tga.TorchGeneratorAgent):
    """
    Example agent.
    Implements the interface for TorchGeneratorAgent. The minimum requirement is that it
    implements ``build_model``, but we will want to include additional command line
    parameters.
    """

    @classmethod
    def add_cmdline_args(
        cls, parser: ParlaiParser, partial_opt: Optional[Opt] = None
    ) -> ParlaiParser:
        """
        Add CLI arguments.
        """
        # Make sure to add all of TorchGeneratorAgent's arguments
        super().add_cmdline_args(parser, partial_opt=partial_opt)

        # Add custom arguments only for this model.
        group = parser.add_argument_group('Example TGA Agent')
        group.add_argument(
            '-hid', '--hidden-size', type=int, default=1024, help='Hidden size.'
        )
        return parser

    def build_model(self):
        """
        Construct the model.
        """

        model = ExampleModel(self.dict, self.opt['hidden_size'])
        # Optionally initialize pre-trained embeddings by copying them from another
        # source: GloVe, fastText, etc.
        self._copy_embeddings(model.embeddings.weight, self.opt['embedding_type'])
        return model

In [ ]:
from parlai.scripts.train_model import TrainModel

TrainModel.main(
    model='my_first_lstm',
    model_file='my_first_lstm/model',
    task='empathetic_dialogues',
    batchsize=1,
    validation_every_n_secs=10,
    max_train_time=6000,
)

21:38:33 | building dictionary first...
21:38:33 | Overriding opt["batchsize"] to 1 (previously: 10)
21:38:33 | Using CUDA
21:38:33 | loading dictionary from my_first_lstm/model.dict
21:38:33 | num words = 22419


TypeError: ignored

In [ ]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # Model files that begin with "zoo:" are special files distributed by the ParlAI team.
    # They'll be automatically downloaded when you ask to use them.
    model_file='my_first_lstm/model'
)

21:31:45 | Using CUDA
21:31:45 | loading dictionary from my_first_lstm/model.dict
21:31:45 | num words = 22419
21:31:45 | Seq2seq: full interactive mode on.
21:31:45 | Total parameters: 39,750,656 (39,750,656 trainable)
21:31:45 | Loading existing model params from my_first_lstm/model
21:31:45 | Opt:
21:31:45 |     adafactor_eps: '[1e-30, 0.001]'
21:31:45 |     adam_eps: 1e-08
21:31:45 |     add_p1_after_newln: False
21:31:45 |     aggregate_micro: False
21:31:45 |     allow_missing_init_opts: False
21:31:45 |     batchsize: 10
21:31:45 |     beam_block_full_context: True
21:31:45 |     beam_block_list_filename: None
21:31:45 |     beam_block_ngram: -1
21:31:45 |     beam_context_block_ngram: -1
21:31:45 |     beam_delay: 30
21:31:45 |     beam_length_penalty: 0.65
21:31:45 |     beam_min_length: 1
21:31:45 |     beam_size: 1
21:31:45 |     betas: '[0.9, 0.999]'
21:31:45 |     bpe_add_prefix_space: None
21:31:45 |     bpe_debug: False
21:31:45 |     bpe_dropout: None
21:31:45 |     bpe

KeyboardInterrupt: ignored